In [57]:
import pandas as pd

df =  pd.read_csv('stockNewsData.csv', encoding='ISO-8859-1')

df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,2000-01-05,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,...,South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,2000-01-06,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,...,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,2000-01-07,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,...,Most everywhere: UDIs,Most wanted: Chloe lunettes,Return of the cane 'completely off the agenda',From Sleepy Hollow to Greeneland,Blunkett outlines vision for over 11s,"Embattled Dobson attacks 'play now, pay later'...",Doom and the Dome,What is the north-south divide?,Aitken released from jail,Gone aloft


In [72]:
# Pre-Processing

# Splitting data into train-test

train = df[df['Date'] <= '20150101']
test = df[df['Date'] >= '20141231']

# Keeping only alphabets and numbers and removing the rest from the data
data = train.iloc[:,2:27]
data.replace('[\W_]+', ' ', regex=True, inplace=True)

# Renaming columns to integer
data.columns = [str(i) for i in range(25)]

# Converting the data to lowercase
for i in data.columns:
    data[i] = data[i].str.lower()
data.head(5)

# For each row, merging all the column data into one paragraph

headlines = []
for i in data.index:
    headlines.append(' '.join(str(a) for a in data.iloc[i,0:25]))

# Applying BOW to the data (converting text to integers)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(2,2))
train_set = cv.fit_transform(headlines)

(3975, 598205)


In [73]:
# Model creation

# Here we will be using Random Forest Classifier for creating a model
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=200, criterion='entropy')
rfc.fit(train_set, train['Label'])

# Predicting on test data
data_test = test.iloc[:,2:27]
data_test.replace('[\W_]+', ' ', regex=True, inplace=True)

test_headlines = []
for i in range(0, len(data_test.index)):
    test_headlines.append(' '.join(str(c) for c in data_test.iloc[i,2:27]))
# Note: We do a cv.transform for test data instead of cv.fit_transform done in train data. This is to ensure test set
# follows the same vocabulary of train set
test_set = cv.transform(test_headlines)
predict = rfc.predict(test_set)

In [75]:
# Model evaluation
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

matrix = confusion_matrix(test['Label'], predict)
print('Confusion Matrix:\n', matrix)

acc_score = accuracy_score(test['Label'], predict)
print('\nAccuracy Score:', acc_score)

class_report = classification_report(test['Label'], predict)
print('\nClassification Report:\n', class_report)

Confusion Matrix:
 [[138  48]
 [ 13 179]]

Accuracy Score: 0.8386243386243386

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.74      0.82       186
           1       0.79      0.93      0.85       192

    accuracy                           0.84       378
   macro avg       0.85      0.84      0.84       378
weighted avg       0.85      0.84      0.84       378

